In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from shapely.geometry import MultiPolygon
from zipfile import ZipFile
import requests 
import sys
%matplotlib inline

In [5]:
import yaml

with open('../../config/postgres.yaml') as f:
    engine_configs = yaml.load(f, Loader=yaml.FullLoader)
    
try:
    engine = create_engine('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(**engine_configs))
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [6]:
CITY = 'boston'

### Neighborhoods

In [8]:
sql = """INSERT INTO spatial_groups (city, core_geom, core_id, lower_ids, spatial_name, approx_geom)
SELECT a.city, a.core_geom, a.core_id, array_agg(a.core_id), 'core', ST_multi(a.core_geom)
FROM spatial_groups a
where a.city='{city}' and a.spatial_name = 'ego'
GROUP BY a.core_id, a.core_geom, a.city;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Land use

In [9]:
land_gdf = gpd.read_file('zip://../../data/boston/land_use/Parcels_2016_Data_Full.zip')

land_gdf['sqftmain'] = land_gdf['LIVING_ARE'].fillna(0) + land_gdf['GROSS_AREA'].fillna(0)

# Remove streets
land_gdf = land_gdf[~((land_gdf["OWNER"]=='COMMONWEALTH OF MASS') & (land_gdf["YR_BUILT"]=='0'))]

land_gdf = land_gdf.rename(columns={'PID_LONG': 'pid', 'LU': 'usecode', 'PTYPE': 'usecode2', 'YR_BUILT': 'yearbuilt', 'AV_TOTAL': 'value'})

land_gdf = land_gdf[['pid', 'sqftmain', 'usecode', 'usecode2', 'yearbuilt', 'geometry', 'value']]

land_gdf = land_gdf[~(land_gdf['geometry'].isnull())]
land_gdf = land_gdf.to_crs({'init': 'epsg:4326'}) 

land_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500


In [10]:
land_gdf_condos = land_gdf[land_gdf['usecode'] == 'Residential Condo Unit'].copy()
land_gdf_condos_geom = land_gdf_condos[['pid', 'geometry']].drop_duplicates(subset='pid')
land_gdf_condos_nogeom = land_gdf_condos.groupby(['pid', 'usecode', 'usecode2', 'yearbuilt'], as_index=False).mean()
land_gdf_condos = pd.merge(land_gdf_condos_geom, land_gdf_condos_nogeom, on='pid')[['pid', 'sqftmain', 'usecode', 'usecode2', 'yearbuilt', 'geometry', 'value']]

land_gdf = pd.concat((land_gdf[land_gdf['usecode'] != 'Residential Condo Unit'], land_gdf_condos))
land_gdf.head()

,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0


In [11]:
wrong_pids = ['0303430000', '0305362000', '0305364000', '0305507000',
       '0305510000', '0305520000', '0306232000', '0400575000',
       '0400592000', '0401141000', '0401082000', '2100396010',
       '0401140000', '0500355000', '0500476000', '0500739001',
       '0500706010', '0501004000', '0501071000', '0501073000',
       '0600001003', '0600002000', '0600002001', '0600004003',
       '0600005000', '0602769010', '0702849000', '0703355000',
       '0703428000', '0703501003', '0703583000', '0703607000',
       '0703677000', '0801021230', '0801070000', '0902819000',
       '1100001000', '1100873000', '1100873010', '1100896030',
       '1101197000', '1101243000', '1102529000', '1102602000',
       '1102618000', '1102638000', '1301107000', '1301199000',
       '1303321001', '1400109100', '1400116100', '1401140100',
       '1401603000', '1402330100', '1402931000', '1700624100',
       '1701590200', '1701604000', '1702190100', '1800734100',
       '1800970000', '1801039000', '1801981000', '1803773000',
       '1804382000', '1805622000', '1806598000', '1806637000',
       '1808197000', '1809226000', '1809477000', '1809793150',
       '1810095000', '1810602100', '1811037000', '1811089000',
       '1811141100', '1811546000', '1812149000', '1812161001',
       '1812738000', '1812974500', '1813019000', '1902798000',
       '1902805001', '1902822000', '1905052010', '2005233000', '0700072010', '0700072030', '2001613500', '2002318500',
       '2002287000', '2005507000', '2005717000', '2005723000',
       '2008811000', '2008871001', '2008917500', '2009155500',
       '2009216300', '2009226000',  '2201868000', '2200105000',
       '2201906100', '2202727000', '2202770000', '2203417000',
       '0103679000', '0103680001', '0103918055', '0103920055',
       '0104165000', '0104166000', '0105346050', '0105346060',
       '0202645000', '0202672000', '0202699000', '0202732000',
       '0202830000', '0202866000', '0202926000', '0203591001',
       '0203591002', '0303048000', '0402123010', '0402133010',
       '0402170010', '0402235200', '0402235300', '0402236100',
       '0402249000', '0402342500', '0504175003', '0504175004',
       '0504175005', '0504198001', '1500574000', '1600113000',
       '1600257000', '1600258000', '1600603000', '1600776000',
       '1601395000', '1601559000', '1602552000', '1602561000',
       '1602563000', '1602723000', '1602775000', '1602784000',
       '1602784001', '1700822000', '1700841000', '1701061000',
       '1701152000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0300944000',
       '0301913000', '0603413000', '0603419001', '0902888000',
       '0902909000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0300448000', '0301130010', '0301133010', '0301136010',
       '0300520000', '0300521000', '0300521010', '0300540000',
       '0300541000', '0300550000', '0301059000', '0301068010',
       '0301072010', '0301074000', '0301121000', '0301121010',
       '0301687000', '0301922000', '0301927050', '0301935003',
       '0302600000', '0302604000', '0303307000', '0303324000',
       '0303435000', '0304981000', '0304991000', '0305361000',
       '0305361003', '0305366000', '0306869000', '0400815000',
       '0401041000', '0502232000', '0502239000', '0502252000',
       '0502495001', '0503043000', '0503704001', '0503704002',
       '0503704003', '0504145000', '0504155000', '0600006010',
       '0600207010', '0600223010', '0600232000', '0600233000',
       '0600240000', '0600248000', '0600257000', '0601245000',
       '0601252010', '0601253000', '0602769013', '0602769017',
       '0602769021', '0700013000', '0700072000', '0700072020',
       '0700072025', '0801024000', '0801024006', '0801024008',
       '0801069000', '1100750000', '1101233040', '1101238000',
       '1101240000', '1102927000', '1102928050', '1102943000',
       '1102944000', '1102945000', '1102946000', '1102947000',
       '1102948000', '1303400000', '1303448000', '1500022000',
       '1500563000', '1500569000', '1600593000', '1600578000',
       '1600748000', '1600756000', '1604320000', '1809440000',
       '1809792050', '1809793010', '1809793100', '1812161000',
       '1812170000', '1812369000', '1812369001', '1813018000',
       '1904603000', '1904604000', '1904606000', '2100063000',
       '2100069015', '2100069035', '2100069050', '2100069055',
       '2100069075', '2100171001', '2100272000', '2100357000',
       '2100357001', '0304995000', '0700073000', '0702812000', '1303259000',
       '1303320000', '1303322000', '0103712000', '0106217000', '0106291000', '0203615000',
       '0203964001', '0203980000', '0301523001', '0301532000',
       '0301570000', '0301927510', '0301910000', '0301911000',
       '0303792000', '0304351000', '0700028001', '0700239000',
       '0702740000', '0702740001', '0703337000', '0703354000',
       '0801404000', '0801415000', '1302240000', '1600225000',
       '1600252000', '1600258001', '1602376001', '1602418000',
       '1602412000', '1602492000', '1602568000', '1602590001',
       '1602735001', '1812603000', '1812736000', '2100396000',
       '2200103000', '0201877000', '0201877001', '0201878001', '0201928000',
       '0201929000', '0202179050', '0202191000', '0203962002',
       '0203962003', '0203962004', '0203962005', '0203962006',
       '0203962007', '0203962008', '0203962009', '0203962020',
       '0203963001', '0203963002', '0400576000', '0500705000',
       '1102642001']
land_gdf = land_gdf[~land_gdf.pid.isin(wrong_pids)]

In [12]:
land_gdf['landuse'] = 'commercial'

land_gdf.loc[(land_gdf['usecode'].str[:1].isin({'A', 'R'})) & (land_gdf['usecode'] != 'Agricultural/Horticultural') & (land_gdf['usecode'] != 'Residential Land'), 'landuse'] = 'residential'
land_gdf.loc[land_gdf['usecode'].isin({'Condominium Master', 'Condominium Parking'}), 'landuse'] = 'residential'
 
land_gdf.loc[(~land_gdf['usecode2'].isin({353, 905, 906})) & (land_gdf['usecode'].isin({'Commercial', 'Industrial', 'Commercial Condo Unit'})) & ((land_gdf['usecode2'] > 383) | (land_gdf['usecode2'] < 360)), 'landuse'] = 'commercial'
land_gdf.loc[(~land_gdf['usecode2'].isin({353, 905, 906})) & ((land_gdf['usecode2'] >= 950) & (land_gdf['usecode2'] <= 965)), 'landuse'] = 'commercial'
land_gdf.loc[(land_gdf['usecode2'] >= 972) & (land_gdf['usecode2'] <= 983), 'landuse'] = 'commercial'

land_gdf.loc[(land_gdf['usecode'] == 'Exempt') & (~((land_gdf['usecode2'] >= 950) & (land_gdf['usecode2'] <= 965))) & (~((land_gdf['usecode2'] >= 972) & (land_gdf['usecode2'] <= 983))), 'landuse'] = 'recreational'
land_gdf.loc[((land_gdf['usecode2'] >= 360) & (land_gdf['usecode2'] <= 383)), 'landuse'] = 'recreational'
land_gdf.loc[land_gdf['usecode2'].isin({353, 905, 906}), 'landuse'] = 'recreational'

land_gdf.loc[(land_gdf['usecode'].isin({'Residential Land', 'Commercial Land'})), 'landuse'] = 'vacant'

# Some fixes
land_gdf.loc[land_gdf['usecode2'].isin({908}), 'landuse'] = 'residential'
land_gdf.loc[land_gdf['usecode2'].isin({904, 902}), 'landuse'] = 'commercial'


mixed = land_gdf[land_gdf['usecode'].isin({'Mixed Use Res/Comm'})].copy()
mixed2 = mixed.copy()

mixed.loc[:, 'landuse'] = 'residential'
mixed.loc[:, 'sqftmain'] /= 2
mixed2.loc[:, 'landuse'] = 'commercial'
mixed2.loc[:, 'sqftmain'] /= 2

land_gdf = pd.concat((land_gdf[~(land_gdf['usecode'].isin({'Mixed Use Res/Comm'}))], mixed, mixed2))
land_gdf.head()

,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value,landuse
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0,residential
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0,residential
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0,residential
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0,residential
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0,residential


In [13]:
# Some mistakes in the dataset
land_gdf.loc[land_gdf.pid == '1902795000', 'landuse'] = 'recreational'
land_gdf.loc[land_gdf.pid == '2005207000', 'landuse'] = 'recreational'

land_gdf.loc[land_gdf.pid == '0104126000', 'landuse'] = 'commercial'

land_gdf.loc[land_gdf.pid.isin(['0104126001',
'0603417000',
'0603411000',
'0603412000',
'0603407000',
'0104446010', '0104447010', '0303027000', '0203505600',
       '0203517700', '0203517800', '0302990000', '0303004000',
       '0303026000', '0303028300', '0303029200', '0303030500',
       '0303035000', '0303036000', '0303036001', '0303036005',
       '0303037000', '0303044000', '0303050000', '0602828001',
       '0602828002', '0602829000', '0305364000', '0602831000',
       '0305365000', '0602642005', '0602673100', '0602672000',
       '0602673001', '0602673002', '0602673002', '0602673002',
       '0602673002', '0602673120', '0602673150', '0602674000',
       '0602674008', '0602674010', '0602674013', '0602674014',
       '0602674015', '0602674019', '0602674020', '0602674025',
       '0602674030', '0602674040', '0602674050', '0602675000',
       '0602675001', '0602676001', '0602676005', '0602676010',
       '0602676025', '0602676035', '0602678000', '0602678020',
       '0602678500', '0602678700', '0602679000', '0602680100',
       '0602680120', '0602680200', '0602683000', '0602746010',
       '0602747010', '0602749100', '0602756020', '0602769030',
       '0602830000', '0602769018', '0602769024', '0602769100',
       '0602769150', '0602774010', '0602782010', '0602798010',
       '0602812100', '0602813000', '0602813003', '0602814000',
       '0602814001', '0602814002', '0602814003', '0602815000',
       '0602815001', '0602816000', '0602817000', '0602817005',
       '0602817020', '0602819001', '0602824000', '0602826000',
       '0602827000', '0602827001', '0602827002', '0602828000',
       '0602834002', '0602835000', '0602836010', '0602838050',
       '0602839005', '0602839010', '0201791000', '0201796000', '0201797001', '0202755004',
       '0202754000', '0202755015', '0202756000', '0202756001',
       '0203594100', '0301925000', '0302573000', '0302605000',
       '0302615000', '0305363100', '0401620000', '0401666000',
       '0600004010', '0600005001', '0600116000', '0600117000',
       '0600118000', '0600119000', '0600121000', '0600123000',
       '0600124000', '0600125000', '0600126000', '0600164000',
       '0600312000', '0801024040', '0801024107', '0801024035',
       '0801024036', '0801024037', '0801834000', '0801854010',
       '0801878010', '0801903010', '0801973000', '0801984010',
       '0802375000', '0802380000', '1400359000', '1400368000', '1405199002', '1800513000',
       '1800968000', '1801034000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0500036000',
       '0300460000', '0300475000', '0300944000', '0301686000',
       '0301913000', '0302722000', '0302723000', '0302714000',
       '0305560010', '0305599010', '0305599030', '0306500000',
       '0306264000', '0306499000', '0401841000', '0401849000',
       '0401850000', '0401996000', '0402011020', '0500010000',
       '0500035000', '0603413000', '0603419001', '0801026000',
       '0801026001', '0801026003', '0801026004', '0801026100',
       '0801027000', '0801027010', '0801027011', '0801027012',
       '0801029100', '0801032005', '0801032010', '0801038002',
       '0801147000', '0801147010', '0801148000', '0801149000',
       '0801150000', '0801151000', '0801153010', '0801167000',
       '0801167020', '0801170010', '0801408000', '0902643000',
       '0902888000', '0902909000', '1001670000', '1001781000',
       '1002199000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0500036000',
       '0300460000', '0300475000', '0300944000', '0301686000',
       '0301913000', '0302722000', '0302723000', '0302714000',
       '0305560010', '0305599010', '0305599030', '0306500000',
       '0306264000', '0306499000', '0401841000', '0401849000',
       '0401850000', '0401996000', '0402011020', '0402043300',
       '0402043000', '0402043100', '0402043200', '0402043400',
       '0402043500', '0402043600', '0402043700', '0500010000',
       '0500035000', '0603413000', '0603419001', '0801026000',
       '0801026001', '0801026003', '0801026004', '0801026100',
       '0801027000', '0801027010', '0801027011', '0801027012',
       '0801029100', '0801032005', '0801032010', '0801038002',
       '0801147000', '0801147010', '0801148000', '0801149000',
       '0801150000', '0801151000', '0801153010', '0801167000',
       '0801167020', '0801170010', '0801408000', '0902643000',
       '0902888000', '0902909000', '1001670000', '1001781000',
       '1002199000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0700072000', '1100750000', '1101233040', '1101238000',
       '1101240000', '1102927000', '1102928050', '1102943000',
       '1102944000', '1102945000', '1102946000', '1102947000',
       '1102948000', '1303400000', '1303448000', '1904603000',
       '1904604000', '1904606000', '0301570000', '0302726500', '0302782000', '0302856000',
       '0302959000', '0303689000', '0303905000', '0304585010',
       '0304590010', '0304995000', '0305229001', '0305231000',
       '0305232000', '0305233000', '0305234000', '0305235000',
       '0305236000', '0305238000', '0305239000', '0305240000',
       '0305242000', '0305246000', '0305248001', '0305249000',
       '0305250000', '0305648000', '0601410000', '0700073000',
       '0702812000', '0802472000', '0802475000', '0903082200',
       '1101465010', '1201261000', '1303259000', '1303320000',
       '1303322000', '1703652000', '1703653000', '1703654000',
       '1703705000', '1800003000', '1800005000', '1801014000', '0202170000', '0203963050', '0203964000', '0302886000',
       '0304585010', '0304590010', '0304890000', '0305648000',
       '0500774000', '1812152010', '2009294000', '2202495000',
       '2202496000', '2202497000']), 'landuse'] = 'commercial'


land_gdf.loc[land_gdf.pid.isin(['0702845000', '0702846000', '0702847000', '0702853000',
       '1800041000', '1800044000', '1800112000', '1800112001',
       '1800113300', '1800113400', '1800113500', '1800257000',
       '1800283010', '1800283020', '1800283030']), 'landuse'] = 'residential'

land_gdf.loc[land_gdf.pid.isin(['1902186000', '1902823000', '1402863000', '1402883000',
                               '1001214000', '0401994001', '0305913000', '0306535000', '0401994001', '0500812000',
       '0501259000', '0502824500', '0502875500', '0502921500',
       '0502939500', '0503371500', '0503391500', '0503564500',
       '0503575500', '0503817500', '0503893500', '0801208000',
       '0801504500', '0801979000', '0801981000', '0802424000',
       '0802922000', '0902025000', '0902025010', '0902025020',
       '0902040000', '0902040010', '0902041000', '0902755000',
       '0903338000', '0903340000', '1000428000', '1000432000',
       '1000434000', '1000435000', '1000436000', '1000437000',
       '1000530000', '1000531000', '1000532000', '1001125000',
       '1001126000', '1001127000', '1001128000', '1001129000',
       '1001130000', '1001131000', '1001132000', '1001133000',
       '1001134000', '1001136000', '1001137000', '1001214000',
       '1001491100', '1202881000', '1402863000', '1402883000',
       '1808199001', '1808200000', '1811040000', '1811041000',
       '1811042000', '1811043000', '1811439000', '1811449000',
       '1811450000', '1900350000', '1900248000', '1900315500',
       '1902186000', '1902823000', '2003590500', '2005121100',
       '2005137000', '2005138000', '2005140000', '2005141000',
       '2005143000', '2005144000', '2005145000', '2005146000',
       '2005147000', '2005148000', '2005149000', '2005150000',
       '2005153000', '2005154000', '2005156000', '2005157000',
       '2005158000', '2005159001', '2005160000', '2005161000',
       '2005162000', '2005163000', '2005309000', '2008967000',
       '2009010000']), 'landuse'] = 'recreational'


In [14]:
land_gdf = land_gdf.reset_index()
land_gdf.head()

,index,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value,landuse
0,0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0,residential
1,1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0,residential
2,2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0,residential
3,3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0,residential
4,4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0,residential


## Net area

In [15]:
unique_land_gdf = land_gdf.copy()
unique_land_gdf.loc[:, 'x'] = unique_land_gdf.geometry.centroid.x
unique_land_gdf.loc[:, 'y'] = unique_land_gdf.geometry.centroid.y
unique_land_gdf = unique_land_gdf.drop_duplicates(subset=['x', 'y'])

In [16]:
ins_gdf = process_geometry_SQL_insert(unique_land_gdf)
ins_gdf.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

NameError: name 'process_geometry_SQL_insert' is not defined

In [ ]:
sql = """
UPDATE temptable_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
FROM
boundary b
WHERE b.city = '{city}' AND ST_Intersects(p.geom, b.geom) AND NOT ST_Touches(p.geom, b.geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [ ]:
sql = """
DELETE 
FROM temptable_{tempname} t
USING unused_areas u 
WHERE u.city = '{city}' AND ST_Intersects(u.geom, t.geom) AND (NOT ST_Touches(u.geom, t.geom)) 
AND (ST_Contains(u.geom, t.geom) OR ST_AREA(ST_Intersection(t.geom, u.geom))/ST_Area(t.geom) > 0.5);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [ ]:
sql = """
INSERT INTO spatial_groups_net_area (sp_id, city, spatial_name, used_area) 
SELECT sp_id, city, spatial_name, SUM(ST_Area(ST_Intersection(s.approx_geom, t.geom)::geography))/1000000.
FROM temptable_{tempname} t
INNER JOIN spatial_groups s ON ST_Intersects(s.approx_geom, t.geom) AND NOT ST_Touches(s.approx_geom, t.geom)
WHERE s.city = '{city}' AND s.spatial_name='core'
GROUP BY sp_id, city, spatial_name;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Refresh materialized views

In [17]:
sql = """
REFRESH MATERIALIZED VIEW spatial_groups_unused_areas;
"""

result = engine.execute(text(sql))